In [1]:
import awkward as ak
import numpy as np
import coffea
import os
from python.plugins import *

print(coffea.__version__)
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import hist

# testFiles = [
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/0CDD30E0-B953-E54B-BB7A-BFB45BE6EC93.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/F6B6A983-3E2B-7B4D-A2CE-E03AB6E32387.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/FDF7FBE4-9226-994A-BD45-29995A3CD62F.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/2CEB03D7-5B1A-EE46-9C21-13ED3E349AF4.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/DB87A517-0584-1A41-BB18-AAE7C9AAE5B8.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/BFE2E7E9-82CB-FE47-9B43-EBA5C373AF60.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5213B536-BAA1-CC43-A58A-B1E41E2680F5.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/E8400F91-3216-EA46-A9F1-8CC7EF5261B0.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root"
# ]
# redirector = 'root://xcache/'
# redirector='root://xrootd-local.unl.edu:1094/'
redirector = 'root://cmsxrootd.fnal.gov/'
# files = handleData('fileset_QCD.json', redirector, testing = True, data = False)
# files_test = handleData('fileset_QCD.json', redirector, testing = False, data = False)
# events = NanoEventsFactory.from_root(redirector+"/store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/C9729B80-15C9-3940-972D-AB1E7610CE6D.root", schemaclass=NanoAODSchema).events()
events = NanoEventsFactory.from_root('root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT1500to2000_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/80000/A0AFD11B-A96C-E049-904C-DC1BD64F3416.root', schemaclass=NanoAODSchema).events()
# print(events.fields)
# print("MC prefiring weights ", events['L1PreFiringWeight'].Nom)
events = events[ak.all(events.FatJet[:,:2].pt > 200., axis = -1)]
print(ak.all(events.FatJet[:,:2].pt > 200., axis = -1))
rho = events.FatJet[:,:2].mass/events.FatJet[:,:2].pt
rho_gen = events.GenJetAK8[:,:2].mass/events.GenJetAK8[:,:2].pt
print("rho max ", np.log(ak.max(rho)), " rho min ", np.log(ak.min(rho)))
print("rho gen max ", ak.max(np.log(rho_gen)), " rho gen min ", ak.min(np.log(rho_gen)))
print(rho)



# import uproot
# # ### uproot notation
# # qcd_mc = uproot.open(list(files.values())[-1][0])
# qcd_mc = uproot.open(redirector+"/store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root")
# uprootevents = qcd_mc['Events']
# print(uprootevents.keys('subjet'))

0.7.22
[True, True, True, True, True, True, True, ... True, True, True, True, True, True]
rho max  -0.60115695  rho min  -4.698739
rho gen max  -0.58963746  rho gen min  -4.666317
[[0.0648, 0.184], [0.128, 0.0753], [0.108, ... [0.222, 0.395], [0.0704, 0.268]]


In [2]:
from python.dijetProcessor import makeDijetHists
# from python.dijetProcessor_old import makeDijetHists
from dask.distributed import Client
import pickle
#### WE'RE MISSING 2016B ver2 -- AK8 PF HLT is missing need to use AK4 trigger isntead
data = False
testing = True
year = "2018"
dask=False
range = None
# mctype="pythiaMG"
mctype="herwig"
winterfell = False
jk=False
jk_range = [0,3]
unc_srcs = ["nominal","HEM", "JER", "JMR", "JMS", "AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1",
"PileUpPtEC2","PileUpPtHF","PileUpPtRef","RelativeFSR","RelativeJEREC1",
            "RelativeJEREC2","RelativeJERHF","RelativePtBB","RelativePtEC1",
            "RelativePtEC2","RelativePtHF","RelativeBal","RelativeSample",
            "RelativeStatEC","RelativeStatFSR","RelativeStatHF","SinglePionECAL",
            "SinglePionHCAL","TimePtEta"]
unc_srcs = ["nominal","HEM", "JER", "JMR", "JMS",]
processor = makeDijetHists(data = data, jet_systematics = unc_srcs, jk=jk) #, systematics = ['LHEPDF']
datastring = "JetHT" if processor.do_gen == False else "QCDsim"
jkstring = ""
if jk:
    if jk_range != None:
        jkstring = "JK" + str(jk_range[0]) + "_" + str(jk_range[1])
    else:
        jkstring = "JK" 
if year == 2016 or year == 2017 or year == 2018:
    year_str = str(year)
elif year == "2016" or year == "2016APV" or year == "2017" or year == "2018":
    year_str = year
else:
    year_str = "All"
if processor.do_gen==True and winterfell:
    filename = "QCD_flat_files.json"
elif processor.do_gen==True:
    # filename = "fileset_QCD.json"
    if mctype == "pythiaMG":
        filename = "fileset_MG_pythia8_wRedirs.json"
    elif mctype == "herwig":
        filename = "fileset_HERWIG_wRedirs.json"
    else:
        filename = "fileset_QCD_wRedirs.json"
else:
    # filename = "datasets_UL_NANOAOD.json"
    filename = "fileset_JetHT_wRedirs.json"
if testing and not data:
    fname = 'coffeaOutput/dijet/dijetHistsTest_{}_fixL1andPU_rapidity{}_{}_{}{}.pkl'.format(datastring, processor.ycut, mctype, jkstring,year_str)
elif testing and data:
    fname = 'coffeaOutput/dijet/dijetHistsTest{}_fixL1andPU_rapidity{}_{}_{}{}.pkl'.format(datastring, processor.ycut, mctype, jkstring,year_str)
elif not testing and data:
    fname = 'coffeaOutput/dijet/dijetHists_{}_fixL1andPU_rapidity{}_{}_{}{}.pkl'.format(datastring, processor.ycut, jkstring, mctype, year_str)
else:
    fname = 'coffeaOutput/dijet/dijetHists_{}_{}_fixL1andPU_rapidity{}_{}{}.pkl'.format(datastring, processor.ycut, mctype, jkstring, year_str)
if range!=None:
    print("Range input: ", range)
    fname=fname[:-4]+"_"+str(range[0])+"_"+str(range[1])+".pkl"
    print("New ranged fname ", fname)

0.7.22
2.7.2
Data: Gen:  False True


In [3]:
run_ind_srcs = False
if run_ind_srcs == True:
    for src in unc_srcs:
        print("Running for source ", src)
        processor = makeDijetHists(data = data, jet_systematics = [src])
        result = runCoffeaJob(processor, jsonFile = filename, year = year, 
                              testing = testing, dask = dask, data = not processor.do_gen, datasetRange=range)
        if testing and not data:
            fname = 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        elif testing and data:
            fname = 'coffeaOutput/dijet/dijetHistsTest{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        elif not testing and data:
            fname = 'coffeaOutput/dijet/dijetHists_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        else:
            fname = 'coffeaOutput/dijet/dijetHists_wXSscaling_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        if range!=None:
            print("Range input: ", range)
            fname=fname[:-4]+"_"+str(range[0])+"_"+str(range[1])+".pkl"
            print("New ranged fname ", fname)
        with open(fname, "wb") as f:
            pickle.dump( result, f)
else:
    result = runCoffeaJob(processor, jsonFile = filename, year = year,
                      testing = testing, dask = dask, data = not processor.do_gen)
    with open(fname, "wb") as f:
        pickle.dump( result, f)

Running over datasets  dict_keys(['/QCD_HT1000to1500_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT100to200_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT1500to2000_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT2000toInf_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT200to300_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT300to500_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT500to700_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', '/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7

Output()

Output()

Filename:  root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/509E8227-440A-8245-A875-5E8203F30BE6.root
Dataset:  /QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM
extracting corrections from files for Summer19UL18_V5_MC


concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1654, in _work_function
    out = processor_instance.process(events)
  File "/srv/python/dijetProcessor.py", line 228, in process
    corrected_fatjets = GetJetCorrections(FatJet, events_jk, era, IOV, isData=not self.do_gen)
  File "/srv/python/corrections.py", line 259, in GetJetCorrections
    ext.add_weight_sets([
  File "/opt/conda/lib/python3.9/site-packages/coffea/lookup_tools/extractor.py", line 99, in add_weight_sets
    self.import_file(thefile)
  File "/opt/conda/lib/python3.9/site-packages/coffea/lookup_tools/extractor.py", line 132, in import_file
    self._filecache[thefile] = file_converters[theformat][thetype](thefile)
  File "/opt/conda/lib/python3.9/site-packages/coffea/lookup_tools/txt_converters.py", line 249, in _convert_standard_jme_txt_file
    return _build_standard_jme_lookup(*_parse_jme_formatted

Filename:  root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/509E8227-440A-8245-A875-5E8203F30BE6.root
Dataset:  /QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM
extracting corrections from files for Summer19UL18_V5_MC
Filename:  root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/509E8227-440A-8245-A875-5E8203F30BE6.root
Dataset:  /QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM
extracting corrections from files for Summer19UL18_V5_MC
Filename:  root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT500to700_TuneCH3_13TeV-madgraphMLM-herwig7/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/50

Exception: Failed processing file: WorkItem(dataset='/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM', filename='root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCH3_13TeV-madgraphMLM-herwig7/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/509E8227-440A-8245-A875-5E8203F30BE6.root', treename='Events', entrystart=0, entrystop=110872, fileuuid=b'\x10\xc0\xab$]\xdb\x11\xed\x98\x90\x074\x9e\x86\xbe\xef', usermeta={})

In [ ]:
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
%matplotlib inline

plt.rcParams["figure.figsize"] = (10,10)

df = pd.DataFrame.from_dict(result['cutflow'])
display(df)
# print(df.keys())
# for key in df.keys():
#     display(df[key])

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Ungroomed MC jets')

result['response_matrix_u'][{'syst':"nominal"}].project('ptreco').plot1d(ax=axs[0,0], binwnorm=True)
result['response_rho_u'].project('rhogen').plot1d(ax=axs[0,1])
result['response_rho_u'].project('rhoreco').plot1d(ax=axs[1,0], binwnorm=True)
result['response_matrix_u'].project('mreco').plot1d(ax=axs[1,1])

for ax in axs.flatten():
    ax.set_xlim(0,1000)

In [ ]:
s = result['response_matrix_u'].stack("syst")
s.project('ptreco').plot(binwnorm=True)
plt.legend()
plt.show()

In [ ]:
fname = "coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_nominal2018.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
%matplotlib inline
#import mplhep as hep
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
fig, axs = plt.subplots(2, 2)
result['ptreco_mreco_u'].project('ptreco').plot1d(ax=axs[0,0], binwnorm=True)
result['ptreco_mreco_u'].project('mreco').plot1d(ax=axs[0,1])
result['ptreco_mreco_g'].project('ptreco').plot1d(ax=axs[1,0], binwnorm=True)
result['ptreco_mreco_g'].project('mreco').plot1d(ax=axs[1,1])

In [ ]:
fig, axs = plt.subplots(2, 2)
for ax in axs.flatten():
    ax.set_xlim([0,1000])
fig.suptitle('Ungroomed Gen jets')
result['ptgen_mgen_u'].project('ptgen').plot1d(ax=axs[0,0], binwnorm=True)
result['ptgen_mgen_u'].project('mgen').plot1d(ax=axs[0,1])
result['ptgen_mgen_g'].project('ptgen').plot1d(ax=axs[1,0], binwnorm=True)
result['ptgen_mgen_g'].project('mgen').plot1d(ax=axs[1,1])

In [ ]:
# numerator = result['jet_pt'][{'partonFlav':'Gluon'}].project('ptreco')
# denominator = result['jet_pt'].project('ptreco')
# print(numerator.values(), denominator.values())

# numerator.label = r'$\epsilon$'
# fig = plt.gcf()
# grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

# main_ax = fig.add_subplot(grid[0])
# subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
# plt.setp(main_ax.get_xticklabels(), visible=False)
# main_ax, sublot_ax = numerator.plot_ratio(
#     denominator,
#     rp_num_label="gluons",
#     rp_denom_label="all",
#     rp_uncert_draw_type="line",
#     rp_uncertainty_type="efficiency",
#     ax_dict = {"main_ax":main_ax,"ratio_ax":subplot_ax}
# )

# plt.xlabel("Pt (GeV)")
# plt.ylabel("Gluon fraction")
# fig.suptitle('Gluon content as a function of pT')

In [ ]:
# numerator = result['jet_eta'][{'partonFlav':'Gluon'}].project('eta')
# denominator = result['jet_eta'].project('eta')
# print(numerator.values(), denominator.values())
# numerator.label = r'$\epsilon$'
# numerator.label = r'$\epsilon$'
# fig = plt.gcf()
# grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

# main_axis = fig.add_subplot(grid[0])
# ratio_ax = fig.add_subplot(grid[1], sharex=main_axis)
# plt.setp(main_axis.get_xticklabels(), visible=False)
# main_ax, sublot_ax = numerator.plot_ratio(
#     denominator,
#     rp_num_label="gluons",
#     rp_denom_label="all",
#     rp_uncert_draw_type="line",
#     rp_uncertainty_type="efficiency",
#     ax_dict = {"main_ax":main_axis,"ratio_ax":ratio_ax}
# )

# plt.xlabel(r'$\eta$')
# plt.ylabel("Gluon fraction")
# main_axis.set_title('Gluon content as a function of eta')


In [ ]:
# #sum entries in bins --> plot as y-axis
# plt.rcParams["figure.figsize"] = (20,5)


# fig, axs = plt.subplots(1, 3)
# result['jet_mass'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")

# fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
# plt.rcParams["figure.figsize"] = (20,10)
# fig, axs = plt.subplots(2, 3)
# print(result['jet_mass'])
# result['jet_mass'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")
# result['jet_mass'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")
# fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
# njets=result['cutflow']['nJets']
# ngluonjets=result['cutflow']['nGluonJets']
# print(result['cutflow'])
# print("Gluon purity:", ngluonjets/njets)

In [ ]:
response_matrix_u_values = result['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").values()
response_matrix_g_values = result['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()

In [ ]:
nptreco,nmassreco,nptgen,nmassgen = response_matrix_u_values.shape

In [ ]:
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )
print(ak.max(response_matrix_u_final))

In [ ]:
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
fig, axs = plt.subplots(2, 2)
axs[0,0].imshow( np.log(response_matrix_u_final+1), aspect="auto", cmap="Blues", interpolation='nearest' )
axs[1,1].imshow( np.log(response_matrix_g_final+1), aspect="auto", cmap="Blues", interpolation='nearest' )
plt.xlabel("RECO", fontsize=20)
plt.ylabel("GEN", fontsize=20)
plt.tick_params(labelsize=20)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( np.log(response_matrix_g_final+1), aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( response_matrix_u_final, aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)